# 品质因数F值（Figure of Merit）
## 计算方法：
\begin{equation}
F = \frac{1}{max(N_{GT}, N_{Seg})}\sum_{i=1}^{N_{Seg}}\frac{1}{1 + \xi \times d_i^2}
\end{equation}

$N_{GT}$为人工检视标注得到的真实的边缘像素点个数 ，$N_{Seg}$ 为通过检测算法检测的边缘像素点个数, $d_i$为检测算法得到的第 个边缘像素点和离它最近的实际边缘像素点间的欧式距离，$\xi$ 为常量系数以惩罚错位的边缘，该系数一般为0.1。F值越大表明检测出的边缘越靠近人工标注的实际边缘, 该边缘检测算法的检测精度越高

In [1]:
import cv2
import numpy as np
import torch
import time


class Fscore():
    """
    本类实现了F值（Figure of Merit）的计算方法，
    引用自:Abdou I E, Pratt W K. Quantitative design and evaluation of enhancement/thresholding edge detectors[J]. 
    Proceedings of the IEEE, 1979, 67(5): 753-763
    """

    def __init__(self, const_index=0.1, gpu_device="cuda:0", verbose=False):
        """
        初始化类
        :param const_index: 常熟，论文中采用0.1
        :param gpu_device: 基于torch的CUDA加速，选择机器
        :param verbose: 是否展示向屏幕打印信息
        """
        self._const_index = 0.1
        self._gpu_device = gpu_device
        self._verbose = verbose

    def get_f_score(self, pred, mask, method=0):
        f_result = 0
        if method == 0:
            f_result = self._calculate_by_torch(pred, mask)
        elif method == 1:
            f_result = self._calculate_by_parallel_matrix(pred, mask)
        elif method == 2:
            f_result = self._calculate_by_neighbor_point(pred, mask)
        return f_score

    def _calculate_by_torch(self, pred, mask):
        """
        针对真值图 mask 和预测图pred计算F值并返回
        先分别计算 pred 和 mask 的坐标x_pred, y_pred, x_mask, y_mask, 假设pred中有m个点，mask中有n个点
        先建成（2，m, n）的矩阵，第一行表示x，第二行表示y, 然后分别是两个方向上，pred减mask的平方
        再将两行加起来形成（m,n），再在axis=1上排序，即为pred中每个点对应mask中最近点的平方,即可求f_score
        该方法由于要生成（2，m, n）三维矩阵，空间消耗巨大，且随着图像变大指数增长，运行时需考虑图像大小
        该方法使用基于Pytorch的GPU加速
        :param pred: 预测图，[0,255]，背景为0，前景为255
        :param mask: 真值图，[0,255]，背景为0，前景为255
        :return: f_score
        """
        start_time = time.time()
        num_pred = np.count_nonzero(pred[pred == 255])
        num_mask = np.count_nonzero(mask[mask == 255])
        num_max = num_pred if num_pred > num_mask else num_mask
        
        pred_tensor = torch.from_numpy(pred)
        mask_tensor = torch.from_numpy(mask)
        start_time = time.time()
        corr_pred = torch.nonzero(pred_tensor == 255)
        corr_mask = torch.nonzero(mask_tensor == 255)
        x_pred = corr_pred[:, 0]
        y_pred = corr_pred[:, 1]
        x_mask = corr_mask[:, 0]
        y_mask = corr_mask[:, 1]     
        end_time = time.time()
        self._print_screen_log("The duration of data transimission to gpu is {}".format(end_time-start_time))
        start_time = time.time()
        dis_xy = torch.zeros(2, num_pred, num_mask).to(self._gpu_device)
        for index in range(0, num_pred):
            dis_xy[0, index, :] = torch.pow(x_mask - x_pred[index], 2)
            dis_xy[1, index, :] = torch.pow(y_mask - y_pred[index], 2)
        end_time = time.time()
        self._print_screen_log("The duration of distance is {}".format(end_time-start_time))
        start_time = time.time()
        dis = dis_xy[0, :, :] + dis_xy[1, :, :]
        dis, _ = dis.sort(1)
        f_score = (torch.sum(1 / (1 + (dis[:, 0] * self._const_index))) / num_max).data
        end_time = time.time()
        self._print_screen_log("The duration of f score is {}".format(end_time-start_time))
        return f_score

    def _calculate_by_parallel_matrix(self, pred, mask):
        """
        针对真值图 mask 和预测图pred计算F值并返回
        先分别计算 pred 和 mask 的坐标x_pred, y_pred, x_mask, y_mask, 假设pred中有m个点，mask中有n个点
        先建成（2，m, n）的矩阵，第一行表示x，第二行表示y, 然后分别是两个方向上，pred减mask的平方
        再将两行加起来形成（m,n），再在axis=1上排序，即为pred中每个点对应mask中最近点的平方,即可求f_score
        该方法由于要生成（2，m, n）三维矩阵，空间消耗巨大，且随着图像变大指数增长，运行时需考虑图像大小
        :param pred: 预测图，[0,255]，背景为0，前景为255
        :param mask: 真值图，[0,255]，背景为0，前景为255
        :return: f_score
        """
        num_pred = np.count_nonzero(pred[pred == 255])
        num_mask = np.count_nonzero(mask[mask == 255])
        num_max = num_pred if num_pred > num_mask else num_mask
        x_pred, y_pred = np.where(pred == 255)
        x_mask, y_mask = np.where(mask == 255)
        start_time = time.time()
        dis_xy = np.zeros((2, num_pred, num_mask))
        for index in range(0, num_pred):
            dis_xy[0, index, :] = np.power(x_mask - x_pred[index], 2)
            dis_xy[1, index, :] = np.power(y_mask - y_pred[index], 2)
        end_time = time.time()
        self._print_screen_log("The duration of distance is {}".format(end_time-start_time))
        start_time = time.time()
        dis = dis_xy[0, :, :] + dis_xy[1, :, :]
        dis.sort(axis=1)
        f_score = np.sum(1 / (1 + (dis[:, 0] * self._const_index))) / num_max
        end_time = time.time()
        self._print_screen_log("The duration of f score is {}".format(end_time-start_time))
        return f_score

    def _calculate_by_neighbor_point(self, pred, mask):
        """
        针对真值图 mask 和预测图pred计算F值并返回
        针对pred中的每个点，遍历其60邻域内最近的mask点,计算d_i,最终计算F_score
        本方法速度最慢
        :param pred: 预测图，[0,255]，背景为0，前景为255
        :param mask: 真值图，[0,255]，背景为0，前景为255
        :return: f_score
        """
        num_pred = np.count_nonzero(pred[pred == 255])
        num_mask = np.count_nonzero(mask[mask == 255])
        num_max = num_pred if num_pred > num_mask else num_mask

        temp = 0.0
        for index_x in range(0, pred.shape[0]):
            for index_y in range(0, pred.shape[1]):
                if pred[index_x, index_y] == 255:
                    distance = self._get_dis_from_mask_point(
                        mask, index_x, index_y)
                    temp = temp + 1 / (1 + self._const_index * pow(distance, 2))
        f_score = (1.0 / num_max) * temp
        return f_score

    def _get_dis_from_mask_point(self, mask, index_x, index_y, neighbor_length=60):
        """
        计算检测到的边缘点与离它最近边缘点的距离
        """

        if mask[index_x, index_y] == 255:
            return 0
        distance = neighbor_length / 2
        region_start_row = 0
        region_start_col = 0
        region_end_row = mask.shape[0]
        region_end_col = mask.shape[1]
        if index_x - neighbor_length > 0:
            region_start_row = index_x - neighbor_length
        if index_x + neighbor_length < mask.shape[0]:
            region_end_row = index_x + neighbor_length
        if index_y - neighbor_length > 0:
            region_start_col = index_y - neighbor_length
        if index_y + neighbor_length < mask.shape[1]:
            region_end_col = index_y + neighbor_length
        # Get the corrdinate of mask in neighbor region
        # becuase the corrdinate will be chaneged after slice operation, we add it manually
        x, y = np.where(mask[region_start_row: region_end_row, region_start_col: region_end_col] == 255)
        
        min_distance = np.amin(np.linalg.norm(np.array([x + region_start_row,y + region_start_col]) - np.array([[index_x], [index_y]]), axis=0))
        
        return min_distance

    def _print_screen_log(self, content):
        """
        像屏幕打印文字，若self._verbose为真，则可打印
        :param content: 打印内容
        """
        if self._verbose:
            print(content)


if __name__ == '__main__':
    image_name = "012_6_4"  # 012_6_4
    mask = cv2.imread("manual_" + image_name + ".png", 0)
    pred = cv2.imread("result_" + image_name + ".png", 0)
    f_score = Fscore(verbose = True)

    print("0. F score weitght By torch gpu")
    start_time = time.time()
    f_result_0 = f_score._calculate_by_torch(pred, mask)
    end_time = time.time()
    print("The value is {}, total Duration is {}".format(f_result_0, end_time-start_time))

    print("1. F score weitght By parallel_matrix")
    start_time = time.time()
    f_result_1 = f_score._calculate_by_parallel_matrix(pred, mask)
    end_time = time.time()
    print("The value is {}, total Duration is {}".format(f_result_1, end_time-start_time))

    print("2. F score weitght By neighbor_point")
    start_time = time.time()
    f_result_2 = f_score._calculate_by_neighbor_point(pred, mask)
    end_time = time.time()
    print("The value is {}, total Duration is {}".format(f_result_2, end_time-start_time))

    if f_result_1 - f_result_2 < 1:
        print("经验证，上述3种方法计算结果相同，结果正确")
    else:
        print("经验证，上述3种方法计算结果不同，结果错误，请排查")

0. F score weitght By torch gpu
The duration of data transimission to gpu is 0.005228519439697266
The duration of distance is 9.97947883605957
The duration of f score is 0.10139870643615723
The value is 0.894863486289978, total Duration is 10.088612794876099
1. F score weitght By parallel_matrix
The duration of distance is 0.5783083438873291
The duration of f score is 1.378657341003418
The value is 0.894863520660748, total Duration is 1.9981017112731934
2. F score weitght By neighbor_point
The value is 0.8948635206607568, total Duration is 0.4526071548461914
经验证，上述3种方法计算结果相同，结果正确
